### Python scratch code of decision tree argorism with the article.
https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

## 1. Gini Index

The Gini index is the name of the cost function used to evaluate splits in the dataset.   
A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5 (for a 2 class problem).   
   
#### proportion   
**&emsp;proportion = count(class_value) / count(rows)**

#### gini index
**&emsp;gini_index = sum(proportion * (1.0 - proportion))**   
**&emsp;gini_index = (1.0 - sum(proportion * proportion)) * (group_size/total_samples)**

In [ ]:
dddd

In [1]:
## example
# group_1_class_0 = 2 / 2 = 1
# group_1_class_1 = 0 / 2 = 0
# group_2_class_0 = 0 / 2 = 0
# group_2_class_1 = 2 / 2 = 1

# Gini(group_1) = (1 - (1*1 + 0*0)) * 2/4
# Gini(group_1) = 0.0 * 0.5 
# Gini(group_1) = 0.0 
# Gini(group_2) = (1 - (0*0 + 1*1)) * 2/4
# Gini(group_2) = 0.0 * 0.5 
# Gini(group_2) = 0.0

### calculate gini index

In [2]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
# 	print("n_instances {}".format(n_instances))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
# 		print("group {}".format(group))
# 		print("classes {}".format(classes))        
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# test Gini values
# print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1])) 
print(gini_index([[[1, 0], [1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1])) # perfect 0

0.0


In [3]:
# gini index calculation example 3
a = 10
b = 15
total = a + b

# all same
gini1 = 2 * (a/total)*(b/total) ## when two group
gini2 = (a/total)*(1-(a/total)) + (b/total)*(1-(b/total))
gini3 = 1 - ((a/total)*(a/total) + (b/total)*(b/total))
print(gini1,gini2,gini3)

0.48 0.48 0.48


## 2. Split

### Split test

In [4]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

### Find best split point

In [5]:
# Select the best split point for a dataset.
# Use two iteration, calculate all possible case. All x variables * all rows.
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset)) # All y value
	b_index, b_value, b_score, b_groups = 999, 999, 999, None # init value
	for index in range(len(dataset[0])-1): # all x variable
		for row in dataset: # all value of x
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score: # update gini index, when it smaller than before(init value is 999)
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

### example

In [6]:
# sample data
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

# get best point
split = get_split(dataset)

# result
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 6.642]


### terminal node value

In [7]:
# Create a terminal node value.
# the most common output value in a list of rows.
def to_terminal(group):
	outcomes = [row[-1] for row in group]
# 	print("outcomes",outcomes)
	return max(set(outcomes), key=outcomes.count) 

### Create child splits for a node or make terminal

In [8]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return None # stop loop
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return None # stop loop
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

### Build a decision tree

In [9]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

### Print tree

In [10]:
# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

### example

In [11]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]


In [12]:
# example of tree / type is dict
tree

{'index': 0,
 'value': 6.642287351,
 'left': {'index': 0,
  'value': 2.771244718,
  'left': 0,
  'right': {'index': 0, 'value': 2.771244718, 'left': 0, 'right': 0}},
 'right': {'index': 0,
  'value': 7.497545867,
  'left': {'index': 0, 'value': 7.444542326, 'left': 1, 'right': 1},
  'right': {'index': 0, 'value': 7.497545867, 'left': 1, 'right': 1}}}

### Make a Prediction

In [13]:
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [14]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

#  predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
	prediction = predict(stump, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Banknote Case Study

In [15]:
# CART on the Bank Note dataset
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
	file = open(filename, "rt")
	lines = reader(file)
	dataset = list(lines)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip()) # remove space and convert string to float

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, []) # shrink dimension
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [16]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count) # the most value in the outcomes. it is a prediction at the test.

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict): # when more child node then do more predit
			return predict(node['left'], row)
		else:
			return node['left'] # end node
	else:
		if isinstance(node['right'], dict): # when more child node then do more predit
			return predict(node['right'], row)
		else:
			return node['right'] # end node

# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)

### excute modeling

In [17]:
# Test CART on Bank Note dataset
seed(1)

# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)

In [18]:
# convert string attributes to integers
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)

In [19]:
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)

In [20]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%


# end of document
------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
len(dataset)

1372

In [25]:
dataset[0:3]

[[3.6216, 8.6661, -2.8073, -0.44699, 0.0],
 [4.5459, 8.1674, -2.4586, -1.4621, 0.0],
 [3.866, -2.6383, 1.9242, 0.10645, 0.0]]

In [24]:
set(a[-1] for a in dataset)

{0.0, 1.0}

### cross validation

In [26]:
# Load a CSV file
def load_csv(filename):
	file = open(filename, "rt")
	lines = reader(file)
	dataset = list(lines)
	return dataset

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy)) # dataset num, get a random number
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [27]:
# Test CART on Bank Note dataset
seed(1)

# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)

In [41]:
dataset[0]

['3.6216', '8.6661', '-2.8073', '-0.44699', '0']

In [40]:
len(dataset)

1372

In [28]:
dt_sp = cross_validation_split(dataset, 5)

In [34]:
for i in range(5):
    print(len(dt_sp[i]))

274
274
274
274
274


In [53]:
randrange(len(dataset))

1082

In [54]:
dataset_copy = list(dataset)

In [58]:
dataset[0:3]

[['3.6216', '8.6661', '-2.8073', '-0.44699', '0'],
 ['4.5459', '8.1674', '-2.4586', '-1.4621', '0'],
 ['3.866', '-2.6383', '1.9242', '0.10645', '0']]

In [59]:
dataset_copy[0:3]

[['3.6216', '8.6661', '-2.8073', '-0.44699', '0'],
 ['4.5459', '8.1674', '-2.4586', '-1.4621', '0'],
 ['3.866', '-2.6383', '1.9242', '0.10645', '0']]

In [87]:
randrange(3)

2

In [88]:
a = [0,1,2,3]

In [89]:
a.pop(2)

2

In [90]:
a

[0, 1, 3]

In [91]:
a.pop(2)

3

In [92]:
a

[0, 1]

### k fold

In [94]:
len(dt_sp)

5

In [100]:
len(dt_sp[0])

274

In [101]:
274*4

1096

In [96]:
# folds = cross_validation_split(dataset, n_folds)
scores = list()
for fold in dt_sp:
	train_set = list(dt_sp)
	train_set.remove(fold)
	train_set = sum(train_set, [])
	test_set = list()
	for row in fold:
		row_copy = list(row)
		test_set.append(row_copy)
		row_copy[-1] = None

In [98]:
len(train_set)

1096

In [103]:
len(test_set)

274

In [104]:
train_set = list(dt_sp)

In [105]:
train_set.remove(fold)

In [106]:
len(train_set)

4

In [107]:
train_set = sum(train_set, [])

In [108]:
len(train_set)

1096

In [110]:
a = [[1,2,3],[4,5,6]]

In [111]:
b = sum(a,[])

In [112]:
b

[1, 2, 3, 4, 5, 6]

### Convert string column to float

In [ ]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [116]:
type(dataset[0][0].strip())

str

In [117]:
type(dataset[0][0])

str